In [ ]:
import numpy as np
from IPython.display import Image
from set_params import *
import glob
from matplotlib import cm
import matplotlib.cm as mpcm
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from getdist import plots, MCSamples
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns

import getdist

from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import scipy

In [ ]:
NN_models = []
NN_settings = '_2LR0.1_128_128'
training_data = N_draws*n_s
loss_funct = str(loss_function[0])
NN_filename = loss_funct +'_ns%s'%training_data + NN_settings

for i in bs:
    for j in lr:
        for w in l2:
            NN_models.append(NN_filename+ '_bs%s'%i + '_lr%s'%j + '_l2%s'%w)
        for k in l1:
            NN_models.append(NN_filename+ '_bs%s'%i + '_lr%s'%j + '_l1%s'%k)
#print(NN_models)

if selection == False:
    file_stan = 'STAN_results/multi_rnd_3D/multirnd_STAN_samples.npy'
else:
    file_stan = 'STAN_results/multi_rnd_3D_selection/multirnd_STAN_samples.npy'
stan = np.load(file_stan)

In [ ]:
def spline_fit(x,y,guess):
    f = interp1d(x, -y, kind='cubic')
    minimum = scipy.optimize.minimize(f,guess)
    x_minimum = minimum['x']
    y_minimum = minimum['fun']
    #x_fit = np.linspace(60,80,10000)
    #y_fit = -f(x_fit)
    #plt.plot(x_fit,y_fit)
    #plt.show()
    return x_minimum, y_minimum

In [ ]:
n_realizations = 100
names = ["H_0","q_0"]
labels = ["H_0","q_0"]

fit_grid = [np.linspace(30,110,1000),np.linspace(-3.5,2.5,100)]

theta = np.loadtxt('{}'.format(pathtorealsamples)+'/multi_thetas{}.txt'.format(ref_filename))[:n_realizations,:]

diff_matrix = np.zeros((len(NN_models),n_realizations,n_params))   #difference between stan and LFI peaks
confidence_matrix = np.zeros((len(NN_models),n_realizations,n_params,2))   #difference between stan and LFI peaks

for j in range (len(NN_models)):
    
    print(NN_models[j])
                       
    for i in range (n_realizations):
        
        stan_samples = stan[i]
        
        posterior_samples = np.loadtxt('LFI_results/results_pydelfi_{}'.format(i) \
                                       + '/results_pydelfi_{}/final_posterior.txt'.format(NN_models[j]))
  
        samples_pydelfi = MCSamples(samples=[posterior_samples],\
                                    names = names, labels = labels)   
        
        for k in range (n_params):
        
            obj = samples_pydelfi.get1DDensity(names[k])
            x_minimum,y_minimum = spline_fit(fit_grid[k],obj.Prob(fit_grid[k]),guess = theta[i,k])
            diff_matrix[j,i,k] = np.mean(stan_samples[:,k]) - x_minimum
            confidence_matrix[j,i,k,:] = obj.getLimits([0.68])[:2]

In [ ]:
mean_matrix = np.zeros((len(NN_models),n_params))
std_matrix = np.zeros((len(NN_models),n_params))

for j in range (len(NN_models)):
    for k in range (n_params):
        mean_matrix[j,k] = np.mean(diff_matrix[j,:,k])
        std_matrix[j,k] = np.std(diff_matrix[j,:,k])

In [ ]:
sigma_matrix_l = np.zeros((len(NN_models),n_realizations,n_params))
sigma_matrix_r = np.zeros((len(NN_models),n_realizations,n_params))
sigma_matrix = np.zeros((len(NN_models),n_realizations,n_params))
diff_sigma_matrix = np.zeros((len(NN_models),n_realizations,n_params))

for j in range (len(NN_models)):    
    
    for i in range (n_realizations):
        
        stan_samples = stan[i]
        
        for k in range (n_params):
        
            x_minimum = np.mean(stan_samples[:,k]) - diff_matrix[j,i,k]
            sigma_matrix_l[j,i,k] = x_minimum-confidence_matrix[j,i,k,0]
            sigma_matrix_r[j,i,k] = confidence_matrix[j,i,k,1]-x_minimum
            sigma_matrix[j,i,k] = (sigma_matrix_l[j,i,k]+sigma_matrix_r[j,i,k])/2
            
            diff_sigma_matrix[j,i,k] = sigma_matrix[j,i,k]/np.std(stan_samples[:,k])

mean_sigma_matrix = np.zeros((len(NN_models),n_params))
std_sigma_matrix = np.zeros((len(NN_models),n_params))

for j in range (len(NN_models)):
    for k in range (n_params):
        mean_sigma_matrix[j,k] = np.mean(diff_sigma_matrix[j,:,k])
        std_sigma_matrix[j,k] = np.std(diff_sigma_matrix[j,:,k])
    #print(mean_sigma_matrix[j,0], 'r$\pm$', std_sigma_matrix[j,0])
    


min_std = 1.5

for i in range (n_realizations):
        
    stan_samples = stan[i]        
    std_stan = np.std(stan_samples[:,0])
    if std_stan < min_std:
        min_std = std_stan
        
std = 0
for i in range (n_realizations):
        
    stan_samples = stan[i]        
    std_stan = np.std(stan_samples[:,0])
    std += std_stan
        
stan_mean_sigma = std/100
print(stan_mean_sigma)

percentage_increase = np.zeros((len(NN_models)))

for j in range (len(NN_models)):
    percentage_increase[j] = np.sqrt((mean_sigma_matrix[j,0]*stan_mean_sigma)**2+(std_matrix[j,0])**2)\
                                /stan_mean_sigma - 1
    percentage_increase[j] = 100*percentage_increase[j]
#    print(percentage_increase[j])

In [ ]:
k = 0
for i in bs:
    for j in lr:
        for w in l2:
            if w==0:
                noreg = 1
            else:
                noreg = 0
            H_string = '%s' %np.round(mean_matrix[k,0],3) + r' \pm %s' %np.round(std_matrix[k,0],3)
            H_sigma_string = '%s' %np.round(mean_sigma_matrix[k,0],3) + r' \pm %s' %np.round(std_sigma_matrix[k,0],3)
            q_string = '%s' %np.round(mean_matrix[k,1],3) + r' \pm %s' %np.round(std_matrix[k,1],3)
            q_sigma_string = '%s' %np.round(mean_sigma_matrix[k,1],3) + r' \pm %s' %np.round(std_sigma_matrix[k,1],3)
            increase_string = '%s' %np.round(percentage_increase[k],2) + r'\%'
            k+=1
            
            if noreg == 1 :
                print('   $'+ str(i) +'$ & $'+ str(j) +'$ & None ' +\
                 ' & $'+ H_string +'$ & $'+ q_string +'$ & $'+ H_sigma_string +'$ & $'+ q_sigma_string +\
                  '$ & $'+ increase_string +'$'+'\\'+'\\')
            
            else:
                print('   $'+ str(i) +'$ & $'+ str(j) +'$ & $\lambda_2 = '+ str(w) +'$ '+\
                 ' & $'+ H_string +'$ & $'+ q_string +'$ & $'+ H_sigma_string +'$ & $'+ q_sigma_string +\
                      '$ & $'+ increase_string +'$'+'\\'+'\\')
            
        for a in l1:
            H_string = '%s' %np.round(mean_matrix[k,0],3) + r' \pm %s' %np.round(std_matrix[k,0],3)
            H_sigma_string = '%s' %np.round(mean_sigma_matrix[k,0],3) + r' \pm %s' %np.round(std_sigma_matrix[k,0],3)
            q_string = '%s' %np.round(mean_matrix[k,1],3) + r' \pm %s' %np.round(std_matrix[k,1],3)
            q_sigma_string = '%s' %np.round(mean_sigma_matrix[k,1],3) + r' \pm %s' %np.round(std_sigma_matrix[k,1],3)
            increase_string = '%s' %np.round(percentage_increase[k],2) + r'\%'
            k+=1       
        
            print('   $'+ str(i) +'$ & $'+ str(j) +'$ & $\lambda_1 = '+ str(a) +'$ '+\
                 ' & $'+ H_string +'$ & $'+ q_string +'$ & $'+ H_sigma_string +'$ & $'+ q_sigma_string +\
                      '$ & $'+ increase_string +'$'+'\\'+'\\')

In [ ]:
k = 0
for i in bs:
    for j in lr:
        min_std = 1.5
        for w in l2:
            if std_matrix[k,0]<min_std:
                index_min = w
                l2_min = 1
                min_std = std_matrix[k,0]
                if w==0:
                    noreg = 1
                else:
                    noreg = 0
                H_string = '%s' %np.round(mean_matrix[k,0],3) + r' \pm %s' %np.round(std_matrix[k,0],3)
                H_sigma_string = '%s' %np.round(mean_sigma_matrix[k,0],3) + r' \pm %s' %np.round(std_sigma_matrix[k,0],3)
                q_string = '%s' %np.round(mean_matrix[k,1],3) + r' \pm %s' %np.round(std_matrix[k,1],3)
                q_sigma_string = '%s' %np.round(mean_sigma_matrix[k,1],3) + r' \pm %s' %np.round(std_sigma_matrix[k,1],3)
                increase_string = '%s' %np.round(percentage_increase[k],2) + r'\%'
            k+=1
            
        for a in l1:
            if std_matrix[k,0]<min_std:
                index_min = a
                l2_min = 0
                min_std = std_matrix[k,0]
                H_string = '%s' %np.round(mean_matrix[k,0],3) + r' \pm %s' %np.round(std_matrix[k,0],3)
                H_sigma_string = '%s' %np.round(mean_sigma_matrix[k,0],3) + r' \pm %s' %np.round(std_sigma_matrix[k,0],3)
                q_string = '%s' %np.round(mean_matrix[k,1],3) + r' \pm %s' %np.round(std_matrix[k,1],3)
                q_sigma_string = '%s' %np.round(mean_sigma_matrix[k,1],3) + r' \pm %s' %np.round(std_sigma_matrix[k,1],3)
                increase_string = '%s' %np.round(percentage_increase[k],2) + r'\%'
            k+=1
            
        if noreg == 1 :
            print('   $'+ str(i) +'$ & $'+ str(j) +'$ & None ' +\
                 ' & $'+ H_string +'$ & $'+ q_string +'$ & $'+ H_sigma_string +'$ & $'+ q_sigma_string +\
                  '$ & $'+ increase_string +'$'+'\\'+'\\')
        else:
            if l2_min == 1:
                print('   $'+ str(i) +'$ & $'+ str(j) +'$ & $\lambda_2 = '+ str(index_min) +'$ '+\
                 ' & $'+ H_string +'$ & $'+ q_string +'$ & $'+ H_sigma_string +'$ & $'+ q_sigma_string +\
                      '$ & $'+ increase_string +'$'+'\\'+'\\')
            else:
                print('   $'+ str(i) +'$ & $'+ str(j) +'$ & $\lambda_1 = '+ str(index_min) +'$ '+\
                 ' & $'+ H_string +'$ & $'+ q_string +'$ & $'+ H_sigma_string +'$ & $'+ q_sigma_string +\
                      '$ & $'+ increase_string +'$'+'\\'+'\\')

training_size = N_draws*n_s
valid_size = N_draws*n_s_val

def add_label(violin, label):
    color = violin["bodies"][0].get_facecolor().flatten()
    labels.append((mpatches.Patch(color=color), label))

fig, ax = plt.subplots(1,bs.shape[0], figsize=(20,10),sharey = 'row'\
                       ,gridspec_kw={'hspace': 0.1, 'wspace': 0.0})
plt.rcParams.update({'font.size': 24})   
  
x_models = []
for j in lr:
    for w in l2:
        if w == 0.0:
            x_models.append('no reg')
        else:
            x_models.append(r' $\lambda_{2}$=' +str(w))
    for k in l1:
        x_models.append(r' $\lambda_{1}$=' + str(k))


ell_loc_legends_0 = [(-0.1,0.8),(0.25,0.8),(0.6,0.8)]
ell_loc_legends_1 = [(-0.1,0.8),(0.25,0.8),(0.6,0.8)]
        

for i in range (bs.shape[0]):
    
    legend_ax1 = [plt.Line2D([0], [0], color=colors[0], lw=0,linestyle='', label=r'$n_{\rm batch}$ = %s          '%bs[i])]
    
    labels = []
    
    data_H = []
    data_q = []
    mean_H = []
    mean_q = []
    x_mod = []
    x = []
    x_H = []
    x_q = []
    
    cc = 0
           
        
    for j in range (i*int(len(NN_models)/bs.shape[0]),(i+1)*int(len(NN_models)/bs.shape[0])):
        if j==0 or j%5 == 0:
            count = 0
            min_std = 1.5
        if j>0 and j%5 == 0:
            cc += 1
        if np.std(diff_matrix[j,:,0])< min_std:
            if count==0:
                data_H.append(diff_matrix[j,:,0])
                data_q.append(diff_matrix[j,:,1])
                mean_H.append(np.mean(diff_matrix[j,:,0]))
                mean_q.append(np.mean(diff_matrix[j,:,0]))
                x_mod.append(x_models[j-i*int(len(NN_models)/bs.shape[0])])
                w = 0.2
                x.append(cc)
                x_H.append(cc - w)
                x_q.append(cc + w)
                count += 1
                min_std = np.std(diff_matrix[j,:,0])
                
            else:
                data_H[-1]=(diff_matrix[j,:,0])
                data_q[-1]=(diff_matrix[j,:,1])
                mean_H[-1]=(np.mean(diff_matrix[j,:,0]))
                mean_q[-1]=(np.mean(diff_matrix[j,:,0]))
                x_mod[-1]=(x_models[j-i*int(len(NN_models)/bs.shape[0])])
                w = 0.2
                x[-1]=(cc)
                x_H[-1] = (cc - w)
                x_q[-1] = (cc + w)
                count += 1
                min_std = np.std(diff_matrix[j,:,0])          

    quartile1, quartile2 = np.percentile(data_H, [16,84], axis=1)
    add_label(ax[i].violinplot(data_H,x_H, showextrema=False, showmeans=True, showmedians=False, widths=w), r'$\Omega : H_0 \;[\mathrm{km\; s^{-1}Mpc^{-1}}]$')
    ax[i].vlines(x_H, quartile1, quartile2, color='royalblue', linestyle='-', lw=1)
    ax[i].scatter(x_H,mean_H)
   
    quartile1, quartile2 = np.percentile(data_q, [16,84], axis=1)
    #ax[i].scatter(x_q,mean_q)
    add_label(ax[i].violinplot(data_q,x_q, showextrema=False, showmeans=True, showmedians=False, widths=w), r'$\Omega : q_0$')
    ax[i].vlines(x_q, quartile1, quartile2, color='red', linestyle='-',lw=1)
    
    
    leg1 = ax[i].legend(handles=legend_ax1,loc='upper center',fontsize=24,frameon=True)
    leg2 = ax[0].legend(*zip(*labels),loc=(0.5,-0.25),fontsize=24,frameon=True, ncol = 2)
    locs = ['upper left','upper center','upper right']
    for j in range (lr.shape[0]):
        legend_lr = [plt.Line2D([0], [0], color=colors[0], lw=0,linestyle='', \
                                label='\n'+'\n'+r' $\alpha$ = %s'%lr[j]+'     ')]
        leg = ax[i].legend(handles=legend_lr,loc=globals()['ell_loc_legends_%s'%i][j],fontsize=24,frameon=False)
        ax[i].add_artist(leg)
    ax[i].add_artist(leg1)
    
    if i == 0:
        ax[i].set_ylabel(r'$b_{\Omega}$',fontsize=26)
    if i == 0:
        if selection == True:  title = r'SELECTION: $[n_{\rm train},n_{\rm val}] = [%s' %training_size +',%s' %valid_size +']$'
        else:    title = r'NO SELECTION: $[n_{\rm train},n_{\rm val}] = [%s' %training_size +',%s' %valid_size +']$'   
        plt.suptitle(title,fontsize = 26)
        
    #ax[i].set_ylim(ymax=0.75,ymin=-0.75)   
    ax[i].hlines(y=0,xmin=x_H[0],xmax=x_q[-1],linestyle=':',alpha=0.5,color='black')
    
    for j in range (len(data_H)):
        if j > 0:
            ax[i].vlines(x=x[j]-0.4,ymin=-1.0,ymax=1.4,linestyle='--',color='black',alpha=0.5)
    #ax[i].vlines(x=2*int(np.max(x)/3)+0.45,ymin=-1,ymax=1,linestyle='--',color='black',alpha=0.5)

    #plt.xlabel('\n'+'architecture',fontsize=26)
    plt.sca(ax[i])
    plt.xticks(x,x_mod,fontsize=24)
ax[0].add_artist(leg2)
plt.show()

In [ ]:
for j in range (len(NN_models)):
    
    print(NN_models[j].split('MSE_ns5000_2LR0.1_128_128_',1)[1])
    print(r'[H_0,q_0] = [%s' %np.round(mean_matrix[j,0],3) + r' ± %s,' %np.round(std_matrix[j,0],2)\
             + ' %s' %np.round(mean_matrix[j,1],3) + r' ± %s]' %np.round(std_matrix[j,1],2))
    
    
    plt.rcParams.update({'font.size': 20}) 
    
    data = {r'$H_0$': theta[:,0],
        r'$q_0$': theta[:,1],
        r'$\bar{H}_{0}^{Stan} - \bar{H}_{0}^{LFI}$': diff_matrix[j,:,0],
        r'$\bar{q}_{0}^{Stan} - \bar{q}_{0}^{LFI}$': diff_matrix[j,:,1]
        }

    df = pd.DataFrame(data,columns=[r'$H_0$',r'$q_0$',r'$\bar{H}_{0}^{Stan} - \bar{H}_{0}^{LFI}$',\
                                r'$\bar{q}_{0}^{Stan} - \bar{q}_{0}^{LFI}$'])
    corrMatrix = df.corr()          
    globals()['corrMatrix_%s'%j] = corrMatrix
    
    f,(ax1,ax2,ax3) = plt.subplots(1,1,sharey=True, figsize=(20,6),gridspec_kw={'width_ratios':[0.92,0.92,1.2]})
    g1 = sns.heatmap(corrMatrix, cbar=False, annot=True,ax=ax1)
    g1.set_ylabel('')
    g1.set_xlabel('')
    g1.set_title('Complete set of test samples')
    #plt.suptitle('Correlation matrices' + '\n ')
    #fig.subplots_adjust(top=2)
    plt.show()

In [ ]:
legend_elements = [plt.Line2D([0], [0], color='w', marker='o',markerfacecolor='blue', label='test data', markersize=10)]


for j in range (len(NN_models)):
    
    print(NN_models[j])
    plt.rcParams.update({'font.size': 24})  
    fig, ax = plt.subplots(2,2,sharey='row',sharex='col',gridspec_kw={'hspace': 0.05,'wspace':0.05}, figsize = (12,10))  
    
    H_mean = np.round(mean_matrix[j,0],3)
    H_std = np.round(std_matrix[j,0],2)
    q_mean = np.round(mean_matrix[j,1],3) 
    q_std = np.round(std_matrix[j,1],2)
    
    #just plotting the original sigma without removing anything
    
    H_fill = [65,75]
    q_fill = [-0.7,-0.3]     
        
    ax[0][0].hlines(y=H_mean,xmin=H_fill[0],xmax=H_fill[1],color='black',linestyle=':')
    ax[0][0].fill_between(H_fill, H_mean-H_std, H_mean+H_std, color='lightgrey',alpha=0.5)
    ax[0][1].hlines(y=H_mean,xmin=q_fill[0],xmax=q_fill[1],color='black',linestyle=':')
    ax[0][1].fill_between(q_fill, H_mean-H_std, H_mean+H_std, color='lightgrey',alpha=0.5)
    ax[1][0].hlines(y=q_mean,xmin=H_fill[0],xmax=H_fill[1],color='black',linestyle=':')
    ax[1][0].fill_between(H_fill, q_mean-q_std, q_mean+q_std, color='lightgrey',alpha=0.5)
    ax[1][1].hlines(y=q_mean,xmin=q_fill[0],xmax=q_fill[1],color='black',linestyle=':')
    ax[1][1].fill_between(q_fill, q_mean-q_std, q_mean+q_std, color='lightgrey',alpha=0.5)

    
    ax[0][0].scatter(theta[:,0],diff_matrix[j,:,0],label='test data')
    ax[1][0].scatter(theta[:,0],diff_matrix[j,:,1])
    ax[0][1].scatter(theta[:,1],diff_matrix[j,:,0])
    ax[1][1].scatter(theta[:,1],diff_matrix[j,:,1])
        
    plt.sca(ax[0, 0])
    plt.ylabel(r'$b_{H_0} \;[\mathrm{km\; s^{-1}Mpc^{-1}}]$')
    plt.sca(ax[1, 0])
    plt.ylabel(r'$b_{q_0}$')
    plt.sca(ax[1, 0])
    plt.xlabel(r'$H_{0} \;[\mathrm{km\; s^{-1}Mpc^{-1}}]$')
    plt.sca(ax[1, 1])
    plt.xlabel(r'$q_{0}$')
    #plt.legend(handles=legend_elements,loc='lower center', bbox_to_anchor=(0., -0.6),fontsize=24,frameon=False, ncol = 3)

    ax[0][0].legend(loc=(0.7,-1.5),frameon=False)
    
    if selection == True:  title = r'SELECTION: $[n_{\rm train},n_{\rm val}] = [%s' %training_size +',%s' %valid_size +']$'
    else:    title = r'NO SELECTION: $[n_{\rm train},n_{\rm val}] = [%s' %training_size +',%s' %valid_size +']$'   
    plt.suptitle(title,fontsize = 26)
    
    plt.show()
    
    if j == len(NN_models)-1:
        fig.savefig("nosel_scatter.pdf", bbox_inches='tight')
    
    print(r'[H_0,q_0] = [%s' %np.round(mean_matrix[j,0],3) + r' ± %s,' %np.round(std_matrix[j,0],2)\
             + ' %s' %np.round(mean_matrix[j,1],3) + r' ± %s]' %np.round(std_matrix[j,1],2) \
         + r' -> [%s' %np.round(mean_matrix_rem[j,0],3) + r' ± %s,' %np.round(std_matrix_rem[j,0],2)\
             + ' %s' %np.round(mean_matrix_rem[j,1],3) + r' ± %s]' %np.round(std_matrix_rem[j,1],2) \
         + r' -> [%s' %np.round(mean_matrix_noedges[j,0],3) + r' ± %s,' %np.round(std_matrix_noedges[j,0],2)\
             + ' %s' %np.round(mean_matrix_noedges[j,1],3) + r' ± %s]' %np.round(std_matrix_noedges[j,1],2))
    